# Fine-Tuning Amazon Nova Micro for Text-to-SQL with Amazon Bedrock Custom Models

This notebook demonstrates how to fine-tune Amazon Nova Micro for text-to-SQL generation using Amazon Bedrock Custom Models. This approach uses Supervised Fine-Tuning (SFT) with Parameter-Efficient Fine-Tuning (PEFT) , SFT is a technique that allows fine-tuning language models on specific tasks using labeled examples, while PEFT enables efficient fine-tuning by updating only a small subset of the model's parameters.

## Overview

This notebook illustrates the complete workflow from data preparation to model evaluation using Bedrock's custom model fine-tuning capabilities:

- **Data Preparation**: SQL dataset converted to bedrock-conversation-2024 schema
- **Training Method**: Bedrock Custom Model fine-tuning with hyperparameter configuration
- **Use Case**: Text-to-SQL generation with comprehensive evaluation and inference pipeline


## Installing Dependencies

In [ ]:
!pip install boto3 datasets pandas scikit-learn --upgrade --quiet

## Setup and Prerequisites

In [ ]:
import boto3
import json
import time
import sagemaker
import io
import os
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from datetime import datetime

# Initialize clients
bedrock = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')
s3_client = boto3.client('s3')
iam_client = boto3.client('iam')

# SageMaker session for S3 operations
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = sess.boto_region_name

print(f"Region: {region}")
print(f"S3 Bucket: {bucket}")
print(f"Role: {role}")

## 1. Data Preparation

We'll use the [sql-create-context](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset and format it according to the bedrock-conversation-2024 schema that Nova expects.

### Step 1a: Load and Explore the Dataset

In [ ]:
# Load the SQL dataset
dataset = load_dataset('b-mc2/sql-create-context', split='train')
print(f'Dataset size: {len(dataset)}')
print('Sample record:')
print(dataset[19])

### Step 1b: Convert to Bedrock Conversation Format

Nova expects data in the bedrock-conversation-2024 format:

```json
{
  "system": [{"text": "System prompt content"}],
  "messages": [
    {
      "role": "user",
      "content": [{"text": "User question"}]
    },
    {
      "role": "assistant",
      "content": [{"text": "Assistant response"}]
    }
  ]
}
```

In [ ]:
def create_bedrock_conversation(record):
    """
    Convert SQL dataset to Bedrock conversation format.
    Format: bedrock-conversation-2024 schema
    """
    system_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question."""
    
    user_message = f"""### Input:
{record['question']}

### Context:
{record['context']}

### Response:
"""
    
    conversation = {
        "system": [{"text": system_prompt}],
        "messages": [
            {
                "role": "user",
                "content": [{"text": user_message}]
            },
            {
                "role": "assistant",
                "content": [{"text": record['answer']}]
            }
        ]
    }
    
    return conversation

# Test the conversion
sample_converted = create_bedrock_conversation(dataset[0])
print('Sample converted record:')
print(json.dumps(sample_converted, indent=2))

### Step 1c: Create Train/Test Split and Convert Dataset

In [ ]:
# Bedrock has a limit of 20,000 training samples
MAX_TOTAL_SAMPLES = 1000  # We will use a smaller subset of data for our finetuning job

# Use 90/10 split, so we need max 20k total samples
total_samples = min(len(dataset), MAX_TOTAL_SAMPLES)

print(f"Original dataset size: {len(dataset)}")
print(f"Using {total_samples} samples (training + validation combined limit: {MAX_TOTAL_SAMPLES})")

# Convert to bedrock-conversation-2024 format
print("\nConverting to Bedrock Format...")
converted_data = []
for i in range(total_samples):
    converted_data.append(create_bedrock_conversation(dataset[i]))
print(f"Converted {len(converted_data)} records")

# Create train/test split (90/10)
train_data, test_data = train_test_split(converted_data, test_size=0.1, random_state=42)

print(f"\nFinal training samples: {len(train_data)}")
print(f"Final test samples: {len(test_data)}")
print(f"Total samples: {len(train_data) + len(test_data)}")
print(f"\n✓ Dataset is within Bedrock's limits (max 20,000 total samples)")

### Step 1d: Save Data in JSONL Format and Upload to S3

In [ ]:
import os

# Save datasets locally
os.makedirs('data', exist_ok=True)

with open('data/train_dataset.jsonl', 'w') as f:
    for item in train_data:
        f.write(json.dumps(item) + '\n')

with open('data/test_dataset.jsonl', 'w') as f:
    for item in test_data:
        f.write(json.dumps(item) + '\n')

print("Datasets saved locally")

# Upload to S3
timestamp = time.strftime('%Y%m%d-%H%M%S')
s3_prefix = f'bedrock-nova-finetuning/{timestamp}'

print("\nUploading to S3...")

# Upload training data
train_key = f'{s3_prefix}/train/train_dataset.jsonl'
try:
    s3_client.upload_file('data/train_dataset.jsonl', bucket, train_key)
    print(f"✓ Uploaded training data to s3://{bucket}/{train_key}")
except Exception as e:
    print(f"✗ Failed to upload training data: {e}")
    raise

# Upload validation data
test_key = f'{s3_prefix}/validation/test_dataset.jsonl'
try:
    s3_client.upload_file('data/test_dataset.jsonl', bucket, test_key)
    print(f"✓ Uploaded validation data to s3://{bucket}/{test_key}")
except Exception as e:
    print(f"✗ Failed to upload validation data: {e}")
    raise

# Set the full S3 paths
train_s3_path = f's3://{bucket}/{train_key}'
test_s3_path = f's3://{bucket}/{test_key}'
output_s3_path = f's3://{bucket}/{s3_prefix}/output/'

print(f"\nS3 Paths:")
print(f"Training data: {train_s3_path}")
print(f"Validation data: {test_s3_path}")
print(f"Output path: {output_s3_path}")

# Verify files exist in S3
print("\nVerifying S3 uploads...")
try:
    response = s3_client.head_object(Bucket=bucket, Key=train_key)
    print(f"✓ Training data verified ({response['ContentLength']} bytes)")
except Exception as e:
    print(f"✗ Training data verification failed: {e}")
    
try:
    response = s3_client.head_object(Bucket=bucket, Key=test_key)
    print(f"✓ Validation data verified ({response['ContentLength']} bytes)")
except Exception as e:
    print(f"✗ Validation data verification failed: {e}")

## 2. Create IAM Role for Bedrock

Create an IAM role that allows Bedrock to access S3 data for training.

In [ ]:
def create_bedrock_role():
    """
    Create IAM role for Bedrock to access S3 data.
    This role allows Bedrock to read training data and write outputs.
    """
    role_name = 'BedrockNovaCustomModelRole'
    
    # Check if role exists
    try:
        response = iam_client.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']
        print(f"Using existing role: {role_arn}")
        return role_arn
    except iam_client.exceptions.NoSuchEntityException:
        pass
    
    # Trust policy for Bedrock
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    # Create role
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Role for Bedrock custom model fine-tuning'
    )
    role_arn = response['Role']['Arn']
    
    # Attach S3 access policy
    s3_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:PutObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{bucket}/*",
                    f"arn:aws:s3:::{bucket}"
                ]
            }
        ]
    }
    
    policy_name = 'BedrockNovaS3Access'
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName=policy_name,
        PolicyDocument=json.dumps(s3_policy)
    )
    
    print(f"Created role: {role_arn}")
    time.sleep(10)  # Wait for role to propagate
    
    return role_arn

bedrock_role_arn = create_bedrock_role()

## 3. Create Bedrock Custom Model Fine-Tuning Job

Now we'll create a fine-tuning job using Bedrock's custom model API.

In [ ]:
print("Starting Bedrock Fine-tuning Job...")

job_name = f'nova-micro-sql-{timestamp}'
custom_model_name = f'nova-micro-sql-custom-{timestamp}'

# Create model customization job
response = bedrock.create_model_customization_job(
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=bedrock_role_arn,
    baseModelIdentifier='arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0:128k', #Base model arn for nova micro 128k context length
    trainingDataConfig={
        's3Uri': train_s3_path
    },
    validationDataConfig={
    'validators': [{
        's3Uri': test_s3_path
    }]
},

    outputDataConfig={
        's3Uri': output_s3_path
    },
    hyperParameters={
        'epochCount': '1',
        'batchSize': '1',
        'learningRate': '0.00001'
    
    },
    customizationType='FINE_TUNING'
)

job_arn = response['jobArn']
print(f"Job ARN: {job_arn}")
print(f"Job Name: {job_name}")
print(f"Custom Model Name: {custom_model_name}")

## 4. Monitor Training Job

Monitor the fine-tuning job until completion. This may take 30-60 minutes.

In [ ]:
def wait_for_customization_job(job_arn, check_interval=60):
    """Monitor the fine-tuning job until completion."""
    print("Monitoring Fine-tuning Job...")
    print("This may take 30-60 minutes depending on dataset size...")
    
    while True:
        response = bedrock.get_model_customization_job(jobIdentifier=job_arn)
        status = response['status']
        
        print(f"Status: {status} - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        if status == 'Completed':
            print("\n✓ Fine-tuning completed successfully!")
            return response['outputModelArn']
        elif status in ['Failed', 'Stopped']:
            failure_message = response.get('failureMessage', 'No failure message provided')
            raise Exception(f"Job {status}: {failure_message}")
        elif status == 'InProgress':
            if 'trainingMetrics' in response:
                metrics = response['trainingMetrics']
                print(f"  Training Loss: {metrics.get('trainingLoss', 'N/A')}")
        
        time.sleep(check_interval)

# Wait for job to complete
custom_model_arn = wait_for_customization_job(job_arn)
print(f"\nCustom Model ARN: {custom_model_arn}")

---
## Wait Until the ^^ Training Job ^^ Completes Above!
---

## 5. Deploy Model for Inference

Deploy the fine-tuned model for on-demand inference.

In [ ]:
#create on-demand inferencing deployment for custom model
def create_model_deployment(custom_model_arn):

    try:
        print(f"Creating on-demand inferencing deployment for model: {custom_model_arn}")
        deployment_name = f'nova-sql-deployment-{timestamp}'
        
        # Create the deployment
        response = bedrock.create_custom_model_deployment(
            modelArn=custom_model_arn,
            modelDeploymentName=deployment_name,
            description=f"finetuned text to sql model: {custom_model_arn}",
        )
        print(response)
        
        # Get the deployment ARN
        deployment_arn = response.get('customModelDeploymentArn')
        
        print(f"Deployment request submitted. Deployment ARN: {deployment_arn}")
        return deployment_arn
    
    except Exception as e:
        print(f"Error creating deployment: {e}")
        return None



In [ ]:
from utils import check_deployment_status

deployment_arn = create_model_deployment(custom_model_arn)

if deployment_arn:
    while True:
        status = check_deployment_status(bedrock, deployment_arn)
        print(f"Model is in {status} phase")
        
        if status == 'Active':
            break
        elif status == 'Failed':
            raise Exception(f"Deployment failed: {deployment_arn}")
        
        time.sleep(15) #sleep for 15 seconds 

print(f"Use the deployment Arn for inferencing: {deployment_arn}")
%store deployment_arn

## 6. Test the Fine-Tuned Model

Test the model with sample queries.

In [ ]:
def invoke_model(model_arn, system_prompt, user_prompt):
    """Invoke the fine-tuned model for inference."""
    request_body = {
        "messages": [
            {
                "role": "user",
                "content": [{"text": user_prompt}]
            }
        ],
        "system": [{"text": system_prompt}],
        "inferenceConfig": {
            "maxTokens": 512,
            "temperature": 0.0
        }
    }
    
    response = bedrock_runtime.converse(
        modelId=model_arn,
        **request_body
    )
    
    return response['output']['message']['content'][0]['text']

print("Testing Fine-tuned Model...")

# Test with a sample from test set
test_sample = test_data[0]
system_prompt = test_sample['system'][0]['text']
user_message = test_sample['messages'][0]['content'][0]['text']
expected_sql = test_sample['messages'][1]['content'][0]['text']

print("Question:")
print(user_message[:200] + "...")
print("\nExpected SQL:")
print(expected_sql)

generated_sql = invoke_model(deployment_arn, system_prompt, user_message)
print("\nGenerated SQL:")
print(generated_sql)

## 7. Comprehensive Evaluation

Run evaluation on multiple test samples.

### Evaluation using an LLM as a judge

Since we have access to the "right" answer, we can evaluate similarity between the SQL queries returned by the fine-tuned Llama model and the right answer. Evaluation can be a bit tricky, since there is no single metric that evaluates semantic and syntactic similarity between two SQL queries. One alternative is to use a more powerful LLM, like Claude 3 Sonnet, to measure the similarity between the two SQL queries (LLM as a judge).

In [ ]:
# Prepare 100 evaluation samples, prompt our fine-tuned model for the sql generation task then ask our judge model to give a score
from utils import ( ask_nova_micro,ask_claude,
    prepare_evaluation_samples,
    test_sql_generation,
    get_score,
    metrics_test,
    run_cold_and_warm_benchmark,
    plot_ttft_comparison
)

eval_samples = prepare_evaluation_samples(test_data, num_samples=100)
# Show a sample
print('\nSample evaluation record:')
print(json.dumps(eval_samples[0], indent=2))

results = test_sql_generation(eval_samples, deployment_arn)

scores = []
print("Grading responses with LLM Judge model")
for result in results:
    if result['status'] == 'success':
        response = float(get_score(
            result['system_prompt'],
            result['query'],
            result['expected_sql'],
            result['generated_sql']
        ))
        scores.append(response)
       

print("Assigned scores: ", scores)
print("The average score of the fine tuned model is: ", sum(scores)/float(len(scores)), "%")


## Operational Metrics for Nova Micro SFT
Now lets test the latency of our Fine tuned Nova Micro LLM by measuring:

* Time To First Token (TTFS) - Cold start time to first token for loading Lora adapters and invoking the model should be is 1 second
* Overall Throughput per Second (OTPS)

In [ ]:
metrics_results= metrics_test(
    model_id= deployment_arn,
    system = "You are a powerful text-to-SQL model. Your job is to answer questions about a database. You can use the following table schema for context: CREATE TABLE table_name_6 (winner_and_score VARCHAR, week VARCHAR)", 
    prompt="Return the SQL query that answers the following question: who is the winner and score for the week of august 9?"
)
print(f"TTFT: {metrics_results['ttft_ms']:.2f}ms")
print(f"OTPS: {metrics_results['otps']:.2f} tokens/s")
print(f"Total end-to-end latency: {metrics_results['total_time_ms']:.2f}ms")

Now lets increase our test cases to get an average result for cold start time as well as warm start time to first token 

In [ ]:
# For quick testing (3 cold starts, 10 warm calls, 2 min wait)

results = run_cold_and_warm_benchmark(
    model_id=deployment_arn,
    system = "You are a powerful text-to-SQL model. Your job is to answer questions about a database. You can use the following table schema for context: CREATE TABLE table_name_6 (winner_and_score VARCHAR, week VARCHAR)", 
    prompt="Return the SQL query that answers the following question: who is the winner and score for the week of august 9?",
    num_cold_starts=5,
    num_warm_calls=10,
    cold_start_wait=600  
)

## Plot the results 

## Use case price comparison analysis 

Below we run an analysis of running a similar workload on a self-hosted ec2 instance as well as a Sagemaker real-time endpoint 
For this analysis we make the following assumptions 

* Users = 100
* Queries per day = 10
* Usage days 30 - 8(weekend) = 22
* Total queries per month = users * queries per day * 22 = 22,000
* Compute hours = 12



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Scenario
users = 100
queries_per_day = 10
total_queries_per_month = users * queries_per_day * 22

# Average tokens per query
avg_input_tokens = 80
avg_output_tokens = 60


In [ ]:

# Bedrock On-Demand 
# ============================================================================

input_cost = (total_queries_per_month * avg_input_tokens / 1000) * 0.000035
output_cost = (total_queries_per_month * avg_output_tokens / 1000) * 0.00014
bedrock_on_demand = input_cost + output_cost

print(f"\n Bedrock On-Demand: ${bedrock_on_demand:.2f}/month")
print(f"   Cost per query: ${bedrock_on_demand/total_queries_per_month:.6f}")

# Self-Hosted on EC2  g5.12xlarge
# ============================================================================

ec2_hourly = 5.672  
ec2_compute = ec2_hourly * 12 * 22

print(f"\n Self-Hosted (EC2 g5.12xlarge): ${ec2_compute:.2f}/month")
print(f"   Cost per query: ${ec2_compute/total_queries_per_month:.4f}")

# SageMaker Endpoint
# ============================================================================

sagemaker_hourly = ec2_hourly+1.418  # (EC2 + SageMaker overhead)
sagemaker_compute = sagemaker_hourly * 12 * 22
sagemaker_total = sagemaker_compute + ec2_hourly

print(f"\n SageMaker Endpoint: ${sagemaker_total:.2f}/month")
print(f"   Compute: ${sagemaker_compute:.2f}")
print(f"   Cost per query: ${sagemaker_total/total_queries_per_month:.4f}")

# ============================================================================
# VISUALIZATION
# ============================================================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Total Monthly Cost
options = ['Bedrock\nOn-Demand', 'Self-Hosted\nEC2', 'SageMaker\nEndpoint']
costs = [bedrock_on_demand, ec2_compute, sagemaker_total]
colors = ['#2ecc71', '#e74c3c', '#f39c12']

bars = ax1.bar(options, costs, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

for bar, cost in zip(bars, costs):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'${cost:,.0f}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax1.set_ylabel('Monthly Cost ($)', fontsize=13, fontweight='bold')
ax1.set_title(f'Monthly Cost Comparison (Verified Pricing)\n{users} users, {queries_per_day} queries/user/day',
             fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Highlight winner
winner_idx = np.argmin(costs)
ax1.get_children()[winner_idx].set_edgecolor('green')
ax1.get_children()[winner_idx].set_linewidth(4)

# Plot 2: Cost per Query
cost_per_query = [c / total_queries_per_month for c in costs]

bars2 = ax2.barh(options, cost_per_query, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

for bar, cpq in zip(bars2, cost_per_query):
    width = bar.get_width()
    ax2.text(width, bar.get_y() + bar.get_height()/2.,
            f' ${cpq:.5f}',
            ha='left', va='center', fontsize=11, fontweight='bold')

ax2.set_xlabel('Cost Per Query ($)', fontsize=13, fontweight='bold')
ax2.set_title('Cost Efficiency', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('cost_comparison_verified.png', dpi=300, bbox_inches='tight')
plt.show()

## Break Even Scale

Now we can see that finetuning our model for on demand usage is significantly cheaper, but at what scale do the other options st

In [ ]:
print("BREAK-EVEN ANALYSIS")

cost_per_query_bedrock = bedrock_on_demand / total_queries_per_month
fixed_self_hosted_costs = ec2_compute

break_even_queries = fixed_self_hosted_costs / cost_per_query_bedrock
break_even_users = break_even_queries / (queries_per_day * 30)

print(f"\nSelf-hosted breaks even at:")
print(f"  {break_even_queries:,.0f} queries/month")
print(f"   = {break_even_users:,.0f} users @ {queries_per_day} queries/day")
print(f"   = {break_even_users/users:.0f}x your current scale")

In [ ]:
#Throughput testing on the deployed fine-tuned model to measure tokens per second, time to first token (TTFT),

from utils import test_model_throughput, visualize_throughput_results
# Run the test

throughput_results = test_model_throughput(deployment_arn)

# Optionally create visualizations
visualize_throughput_results(throughput_results)


### Compare TTFT for Base Nova Micro Model to our SFT Model
Benchmark the Time to First Token (TTFT) performance between the base Amazon Nova Micro model and our supervised fine-tuned (SFT) LoRA model to quantify latency and validate that fine-tuning maintains acceptable response time characteristics while delivering better accuracy for text-to-SQL generation tasks.

In [ ]:
#Becasue the base model does not have knowledge of the SQL data we will ask a generic question to both models

from utils import compare_models

base_model_id = "us.amazon.nova-micro-v1:0"

system_prompt = "You are a helpful assistant."
test_prompt = "What are the performance specs of a bmw x5m and how does it compare with the porsche macan turbo"

results = compare_models(
    custom_model_arn=deployment_arn,
    base_model_id=base_model_id,
    system=system_prompt,
    prompt=test_prompt,
    num_runs=10  
)


In [ ]:
# what is the percentage in latency that the SFT model has in generating TTFT
ttft_percentage_difference = ((381.49-356.28)/356.28)*100
print("Our Custom model has a Time to first token differene of: ", ttft_percentage_difference,"%")
otps_percentage_difference = ((184.56-253.57)/253.57)*100
print("Our Custom model has a Output per second differene of: ", otps_percentage_difference,"%")


---
## Cleanup Resources

**IMPORTANT:** Run this cell to delete all resources created in this notebook to avoid ongoing charges.

In [ ]:
# Cleanup - Delete all resources
import shutil

# Delete provisioned throughput (if created)
try:
    bedrock.delete_custom_model_deployment(customModelDeploymentIdentifier=deployment_arn)
    print("Custom model deleted")
except:
    pass

# Delete custom model
try:
    bedrock.delete_custom_model(modelIdentifier=custom_model_arn)
    print("Custom model deleted")
except:
    pass

# Delete IAM role
try:
    iam_client.delete_role_policy(RoleName='BedrockNovaCustomModelRole', PolicyName='BedrockNovaS3Access')
    iam_client.delete_role(RoleName='BedrockNovaCustomModelRole')
    print("IAM resources deleted")
except:
    pass

# Delete local data
if os.path.exists('data'):
    shutil.rmtree('data')
    print("Local data deleted")

print("\nCleanup complete!")